In [11]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [18]:
# Evaluation
threshold = 0.7

y_true = np.random.rand(32, 256, 128, 1) > threshold
y_pred = np.random.rand(32, 256, 128, 1) > threshold

expected = (np.square(1 - threshold) / (2 * (1 - threshold) - np.square(1 - threshold)) +
            np.square(threshold) / (2 * threshold - np.square(threshold))) / 2

m = tf.keras.metrics.MeanIoU(num_classes=2)

def get_metrics(y_true, y_pred, num_classes=2, metrics='IoU', graph_for=-1):
    """calculate IOU for confidence map of an object"""
    iou = 0
    for k in range(num_classes):
      t = y_true == k
      p = y_pred == k
      i = np.logical_and(t, p)

      if graph_for == k:
        _, ax = plt.subplots(1, 3, figsize=(12, 4))
        ax[0].imshow(t[0, :, :, 0], cmap='gray')
        ax[0].set_title('label')
        ax[1].imshow(p[0, :, :, 0], cmap='gray')
        ax[1].set_title('predict')
        ax[2].imshow(i[0, :, :, 0], cmap='hot')
        ax[2].set_title('intersection')

      i_sum = np.sum(i)
      t_sum = np.sum(t)
      p_sum = np.sum(p)

      if metrics == 'f1':
        iou += 2 * i_sum / (t_sum + p_sum + 1e-7)
      elif metrics == 'IoU':
        u_sum = t_sum + p_sum - i_sum
        iou += i_sum / (u_sum + 1e-7)
      else:
        raise Exception("Invalid metric")

    return iou / num_classes


m.update_state(y_true, y_pred)
iou =  m.result().numpy()
print(f"IoU expected: {expected}, calculated: {iou}")

IoU expected: 0.3574660633484163, calculated: 0.3575238585472107
